# Quick Start Guide to GenAIRR

Welcome to the Quick Start Guide for GenAIRR, a Python module designed for generating synthetic Adaptive Immune Receptor Repertoire (AIRR) sequences. This guide will walk you through the basic usage of GenAIRR, including setting up your environment, simulating heavy and light chain sequences, and customizing your simulations.


## Installation

Before you begin, ensure that you have Python 3.x installed on your system. GenAIRR can be installed using pip, Python's package installer. Execute the following command in your terminal:


In [1]:
import pandas as pd
# Install GenAIRR using pip
#!pip install GenAIRR

## Setting Up

To start using GenAIRR, you need to import the necessary classes from the module. We'll also set up a `DataConfig` object to specify our configuration.


In [ ]:
# Importing GenAIRR classes
from GenAIRR.pipeline import AugmentationPipeline
from GenAIRR.data import HUMAN_IGH_OGRDB, HUMAN_IGL_OGRDB, HUMAN_IGK_OGRDB
from GenAIRR.steps import (SimulateSequence, FixVPositionAfterTrimmingIndexAmbiguity, 
                          FixDPositionAfterTrimmingIndexAmbiguity, FixJPositionAfterTrimmingIndexAmbiguity)
from GenAIRR.steps import (CorrectForVEndCut, CorrectForDTrims, CorruptSequenceBeginning, 
                          InsertNs, InsertIndels, ShortDValidation, DistillMutationRate)
from GenAIRR.mutation import S5F
from GenAIRR.steps.StepBase import AugmentationStep
from GenAIRR.simulation import HeavyChainSequenceAugmentor, LightChainSequenceAugmentor, SequenceAugmentorArguments
from GenAIRR.dataconfig import DataConfig

# Use built-in data configurations directly
heavy_chain_config = HUMAN_IGH_OGRDB
kappa_chain_config = HUMAN_IGK_OGRDB  
lambda_chain_config = HUMAN_IGL_OGRDB

## Simulating Heavy Chain Sequences

Let's simulate a BCR heavy chain sequence using the default GenAIRR pipeline for BCR heavy chain sequences via the `AugmentationPipeline`. This example demonstrates a simple simulation with default settings.


In [3]:
# Set the dataconfig for the simulations
AugmentationStep.set_dataconfig(heavy_chain_config)

# Create the simulation pipeline
pipeline = AugmentationPipeline([
    SimulateSequence(S5F(min_mutation_rate=0.003, max_mutation_rate=0.25), True),
    FixVPositionAfterTrimmingIndexAmbiguity(),
    FixDPositionAfterTrimmingIndexAmbiguity(),
    FixJPositionAfterTrimmingIndexAmbiguity(),
    CorrectForVEndCut(),
    CorrectForDTrims(),
    CorruptSequenceBeginning(0.7, [0.4, 0.4, 0.2], 576, 210, 310, 50),
    InsertNs(0.02, 0.5),
    ShortDValidation(),
    InsertIndels(0.5, 5, 0.5, 0.5),
    DistillMutationRate()
])

# Simulate a heavy chain sequence
heavy_sequence = pipeline.execute()

# Print the simulated heavy chain sequence
print("Simulated Heavy Chain Sequence:", heavy_sequence.get_dict())

Simulated Heavy Chain Sequence: {'sequence': 'GGGGGTCCCTGAGACTCTCCGGGGCAGTGTCTGGATTCACCNTCAGTAGCTATGGCATGCACTGGGTCTNCCAGGCTCCANGCAAGGGGCTGGAGTGGGTGACATTTACAACGGATAAAGGCAGTAATAAATACTATGCAGACTNCGTGAAGGGCCGATCCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATGTTCAAATGAACAGCCTGAGACGTGAGGACACNGCTGTGTTTTACTGGCGAAAGATCCTGACTACACTAGTTCCACCAACTGGTTCCACCACTGGGGCCCGGGAACCCTGGNCACCGTCTCCTCAGGCATCCC', 'v_call': ['IGHVF10-G37*08'], 'd_call': ['IGHD4-4*01', 'IGHD4-11*01'], 'j_call': ['IGHJ5*02'], 'c_call': ['IGHA2*01'], 'v_sequence_start': 0, 'v_sequence_end': 252, 'd_sequence_start': 255, 'd_sequence_end': 266, 'j_sequence_start': 273, 'j_sequence_end': 323, 'v_germline_start': 43, 'v_germline_end': 296, 'd_germline_start': 0, 'd_germline_end': 11, 'j_germline_start': 1, 'j_germline_end': 51, 'junction_sequence_start': 242, 'junction_sequence_end': 292, 'mutation_rate': 0.07575757575757576, 'mutations': {20: 'T>G', 22: 'T>G', 27: 'C>T', 68: 'C>T', 101: 'G>A', 108: 'T>C', 116: 'G>A', 118: 'T>A', 121: 'A>C', 159: 

## Customizing Simulations

GenAIRR allows for extensive customization to closely mimic the natural diversity of immune sequences. Below is an example of how to customize mutation rates and indel simulations.


In [4]:
# Customize augmentation arguments

custom_mutation_model = S5F(min_mutation_rate=0.1,max_mutation_rate=0.5)
custom_insert_indel_step = InsertIndels(indel_probability = 0.05,max_indels = 15,insertion_proba=0.7,deletion_proba=0.3)

pipeline = AugmentationPipeline([
    SimulateSequence(mutation_model = custom_mutation_model,productive = True), # notice here in the simulate sequence step we used the custom mutation model we defined
    FixVPositionAfterTrimmingIndexAmbiguity(),
    FixDPositionAfterTrimmingIndexAmbiguity(),
    FixJPositionAfterTrimmingIndexAmbiguity(),
    CorrectForVEndCut(),
    CorrectForDTrims(),
    CorruptSequenceBeginning(corruption_probability = 0.7,corrupt_events_proba = [0.4,0.4,0.2],max_sequence_length = 576,nucleotide_add_coefficient = 210,
                             nucleotide_remove_coefficient = 310,nucleotide_add_after_remove_coefficient = 50,random_sequence_add_proba = 1,
                             single_base_stream_proba = 0,duplicate_leading_proba = 0,random_allele_proba = 0),
    InsertNs(n_ratio = 0.02,proba = 0.5),
    ShortDValidation(short_d_length= 5),
    custom_insert_indel_step, # notice here we used the custom insert indel step we have created above
    DistillMutationRate()
    ])



# Simulate a heavy chain sequence
heavy_sequence = pipeline.execute()

# Print the simulated heavy chain sequence
print("Customized Simulated Heavy Chain Sequence:", heavy_sequence.get_dict())


Customized Simulated Heavy Chain Sequence: {'sequence': 'CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGGAGCCTTGGGACACCCTGACCCTCACCTCCACTGTCTCTAGTTACCCCATCAGAAATATTATCAGGTTGGACTGGANCCGTCANCCCCCAGTGAAGAGACTGGAGTGGATTGGATACATCGATTATAGAGGGAGCATCAATCACAATCCGTCCCTGAACAGTCGAGTCACCATGTCAGTGGACACGTCCAAGANCCCGTTCTCCCTGAATCTGAACTCTGTGAGCGCCNTGGACACGACCCTGTATTATTGTGCGGGAATCGANTGGGGGGGGGCTTCAGAGAGNCANTACTACGAAATGGACGTCTGGGGCCAAGGGACCACGGTCACCGCCGCCTCAGG', 'v_call': ['IGHVF3-G9*03'], 'd_call': ['IGHD3-10*03'], 'j_call': ['IGHJ6*02'], 'c_call': ['IGHG1*07'], 'v_sequence_start': 0, 'v_sequence_end': 295, 'd_sequence_start': 311, 'd_sequence_end': 320, 'j_sequence_start': 321, 'j_sequence_end': 376, 'v_germline_start': 0, 'v_germline_end': 295, 'd_germline_start': 12, 'd_germline_end': 21, 'j_germline_start': 8, 'j_germline_end': 63, 'junction_sequence_start': 285, 'junction_sequence_end': 345, 'mutation_rate': 0.1246684350132626, 'mutations': {36: 'A>G', 43: 'C>G', 54: 'T>A', 64: 'G>C', 66: 'G>A', 75: 'G>A', 81

## Generating Naïve Sequences

In immunogenetics, a naïve sequence refers to an antibody sequence that has not undergone the process of somatic hypermutation. GenAIRR allows you to simulate such naïve sequences using the `HeavyChainSequence` class. Let's start by generating a naïve heavy chain sequence.


In [5]:
from GenAIRR.sequence import HeavyChainSequence

# Create a naive heavy chain sequence
naive_heavy_sequence = HeavyChainSequence.create_random(data_config_builtin)

# Access the generated naive sequence
naive_sequence = naive_heavy_sequence

print("Naïve Heavy Chain Sequence:", naive_sequence)
print('Ungapped Sequence: ')
print(naive_sequence.ungapped_seq)


Naïve Heavy Chain Sequence: 0|-----------------------------------------------------------------------V(IGHVF10-G42*05)|298336|----------J(IGHJ4*02)|378|378|---------C(IGHG1*08)|416
Ungapped Sequence: 
GAAGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTACAGCCTGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTGATGATTATGCCATGCACTGGGTCCGTCAAGCTCCAGGGAAGGGTCTGGAGTGGGTCTCTCTTATTAGTGGGGATGGTGGTAGCACATACTATGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACAGCAAAAACTCCCTGTATCTGCAAATGAACAGTCTGAGAACTGAGGACACCGCCTTGTATTACTGTGCAAAAGATATCGTAGGGGACTCAATATGGTCTGTGAAGTACAGGCAATTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCACC


## Applying Mutations

To mimic the natural diversity and evolution of immune sequences, GenAIRR supports the simulation of mutations through various models. Here, we demonstrate how to apply mutations to a naïve sequence using the `S5F` and `Uniform` mutation models from the mutations submodule.


### Using the S5F Mutation Model

The `S5F` model is a sophisticated mutation model that considers context-dependent mutation probabilities. It's particularly useful for simulating realistic somatic hypermutations.


In [6]:
from GenAIRR.mutation import S5F

# Initialize the S5F mutation model with custom mutation rates
s5f_model = S5F(min_mutation_rate=0.01, max_mutation_rate=0.05)

# Apply mutations to the naive sequence using the S5F model
s5f_mutated_sequence, mutations, mutation_rate = s5f_model.apply_mutation(naive_heavy_sequence)

print("S5F Mutated Heavy Chain Sequence:", s5f_mutated_sequence)
print("S5F Mutation Details:", mutations)
print("S5F Mutation Rate:", mutation_rate)


S5F Mutated Heavy Chain Sequence: GAAGCGCAGCTGGTGAAGTCTGGGGGAGGCGTGGTACAGCCTGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTGATAATTATGCCATACACTGGGTCCGTCAAGCTCCAGGGAAGGGTCTGGAGTGGGTCTCTCTTTTTACTGGGGATGTTGGTAGAACATACTATGCAGACTCTGCGAAGGGCCGATTCACCATCTCCAGAGACAACAGAAAAAACTCCCTGTATCTGCAAATGAACAGTGTGAGAACTGAGGACAGCGCCTTGTATTACTGTGCAAAAGATATCGTAGGGGACTCACTATGGTCTGTGAAGTACAGGCGATTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGCCTCCACGAAGGGCCCATCGGTCTTCCCCCTGGCACC
S5F Mutation Details: {312: 'A>C', 154: 'G>C', 224: 'C>A', 150: 'A>T', 90: 'G>A', 255: 'C>G', 170: 'C>A', 101: 'G>A', 271: 'C>G', 190: 'T>C', 163: 'G>T', 4: 'T>C', 15: 'G>A', 334: 'A>G', 386: 'C>G'}
S5F Mutation Rate: 0.036575814563128965


### Using the Uniform Mutation Model

The `Uniform` mutation model applies mutations at a uniform rate across the sequence, providing a simpler alternative to the context-dependent models.


In [7]:
from GenAIRR.mutation import Uniform

# Initialize the Uniform mutation model with custom mutation rates
uniform_model = Uniform(min_mutation_rate=0.01, max_mutation_rate=0.05)

# Apply mutations to the naive sequence using the Uniform model
uniform_mutated_sequence, mutations, mutation_rate = uniform_model.apply_mutation(naive_heavy_sequence)

print("Uniform Mutated Heavy Chain Sequence:", uniform_mutated_sequence)
print("Uniform Mutation Details:", mutations)
print("Uniform Mutation Rate:", mutation_rate)


Uniform Mutated Heavy Chain Sequence: GAAGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTTCATCCTGGGGGGTCGCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTGATAATTATGCCACGCACTGGGTCAGTCAAGCTCCGGGGAAGTGTCTGGAGTGGGTCTCTCTTATTAGTGGGGATTGTCGTAGCACATACTATGCAGAGTCTGTGAAGGGCCGATTCACCATCTCCCGAGACAACAGCAAAAACTCCCTGTATCTGCATATGTACAGTCTGAGATCTGAGGACACCGCGTTGTATCACTGTGCAAAAGATATCGTAGGGGACTCAATATGGTCTGTGAAGTACAGGCAATTGACTTCTGGGGCTAGGGAACCCTGGTCACTGTCTCCTCAGGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCACC
Uniform Mutation Details: {38: 'G>T', 165: 'G>C', 111: 'C>A', 213: 'A>C', 185: 'C>G', 282: 'T>C', 245: 'A>T', 342: 'A>T', 122: 'A>G', 100: 'T>C', 249: 'A>T', 275: 'C>G', 261: 'A>T', 162: 'G>T', 90: 'G>A', 367: 'C>T', 350: 'C>T', 50: 'C>G', 35: 'A>T', 129: 'G>T'}
Uniform Mutation Rate: 0.04850801933682698


## Common Use Cases

GenAIRR is a versatile tool designed to meet a broad range of needs in immunogenetics research. This section provides examples and explanations for some common use cases, including generating multiple sequences, simulating specific allele combinations, and more.


### Generating Many Sequences

One common requirement is to generate a large dataset of synthetic AIRR sequences for analysis or benchmarking. Below is an example of how to generate multiple sequences using GenAIRR in a loop.


In [8]:
num_sequences = 5  # Number of sequences to generate

heavy_sequences = []
for _ in range(num_sequences):
    # Simulate a heavy chain sequence
    heavy_sequence = pipeline.execute().get_dict()
    heavy_sequences.append(heavy_sequence)

# Display the generated sequences
for i, seq in enumerate(heavy_sequences, start=1):
    print(f"Heavy Chain Sequence {i}: {seq}")


Heavy Chain Sequence 1: {'sequence': 'TGACGGAATAATCCCTTTCGAGTGGGTTGAAGCCCAGGGAAGAGATCAGACGATTCATTGTTAACAGCTAAGGGTGTGGCGNTTGTGTGGGGGACACGCGATGNCGTTTGGGAGAGGCCNGATCTGGCCGGGAGAGTCCCAGCGAGTCTCTGGTGCAGCTTCTCACTTGACCTTGCGAGGTCTCATATCAGACTGGGTCCNCCGGACTCCNGANAAGGGACTGGCCGGAGTCTCGTCCCGANTTAGGCCAAGCGGTCGCAAGTGGTGCACAAAGGCGCTCACGGGCCCCCTCGTCGTTTCCATAGACTCGGTCAGACGGTCTCTGNCTTCGCAGTTGGAGAGGCGGACGGGCGATGACACAAGTGGNCCGTAATTCTCGTGACTGGGAATGTAACGACGCGGGGCTGAGCCCGACATATGATGTGCTAGGGGCGTCGGGGTCAACGGGACGTCGGACAATGGCCCGCGAGGCCTGCGTCCCGGGCCCGTC', 'v_call': ['IGHVF10-G51*05'], 'd_call': ['IGHD5-5*01', 'IGHD5-18*01'], 'j_call': ['IGHJ6*03'], 'c_call': ['IGHG2*12'], 'v_sequence_start': 88, 'v_sequence_end': 384, 'd_sequence_start': 400, 'd_sequence_end': 407, 'j_sequence_start': 413, 'j_sequence_end': 470, 'v_germline_start': 0, 'v_germline_end': 296, 'd_germline_start': 2, 'd_germline_end': 9, 'j_germline_start': 6, 'j_germline_end': 63, 'junction_sequence_start': 373, 'junction_sequence_end': 439, 'mutation_

In [9]:
import pandas as pd
pd.DataFrame(heavy_sequences)

,sequence,v_call,d_call,j_call,c_call,v_sequence_start,v_sequence_end,d_sequence_start,d_sequence_end,j_sequence_start,...,c_trim_3,productive,stop_codon,vj_in_frame,note,corruption_event,corruption_add_amount,corruption_remove_amount,corruption_removed_section,corruption_added_section
0,TGACGGAATAATCCCTTTCGAGTGGGTTGAAGCCCAGGGAAGAGAT...,[IGHVF10-G51*05],"[IGHD5-5*01, IGHD5-18*01]",[IGHJ6*03],[IGHG2*12],88,384,400,407,413,...,20,False,True,False,Stop codon present.,add,88,0,,TGACGGAATAATCCCTTTCGAGTGGGTTGAAGCCCAGGGAAGAGAT...
1,GATGACGTCAAGAGGTCCCCAGACAAGCGCTTCAGTGAACGTCAGA...,[IGHVF6-G20*02],[IGHD3-3*01],[IGHJ6*02],[IGHG3*26],12,191,196,215,218,...,19,False,True,False,Stop codon present.,remove_before_add,15,115,GACAACCACTTGGCGCAGTGCCGGGCTGTGGATAAAGTGCCTGGAC...,GATGACGTCAAGAGG
2,GCTCTAAACAGTAATTATGCCGTCCGTNCACCAGTCCCCAGGCAAG...,[IGHVF3-G10*03],[IGHD3-22*01],[IGHJ6*02],[IGHE*04],25,208,223,232,233,...,14,False,False,True,V second C not present.,remove_before_add,25,114,TAATTTCATCTGCAGGGGTCGGGCCCAGGGCTAGGGAAGCCTCTCG...,GCTCTAAACAGTAATTATGCCGTCC
3,GAGGTGCAGTTGATGGAGTCGGGGAGTGACTTGGTCCGGCCGGCAG...,[IGHVF10-G49*04],"[IGHD5-18*01, IGHD5-5*01]",[IGHJ3*02],[IGHG3*25],0,297,313,321,324,...,12,False,True,False,Stop codon present.,no-corruption,0,0,,
4,GGGCCTGGGAGTGGGTCTCATCCGATAGTGATGGAAGCACGCAGTG...,[IGHVF10-G35*02],[IGHD1-14*01],[IGHJ6*03],[IGHG1*13],0,164,166,174,180,...,24,False,True,False,Stop codon present.Stop codon present.,remove,0,128,GGGGTGCACCTGGCGGTGTCTGGGGCAGAGTTGGTTCAGCCTGGAG...,


### Generating a Specific Allele Combination Sequence

In some cases, you might want to simulate sequences with specific V, D, and J allele combinations. Here's how to specify alleles for your simulations.


In [13]:
# Define your specific alleles
v_allele = 'IGHVF6-G21*01'
d_allele = 'IGHD5-18*01'
j_allele = 'IGHJ6*03'
c_allele = 'IGHA1*01'

# Extract the allele objects from data_config
v_allele = next((allele for family in data_config_builtin.v_alleles.values() for allele in family if allele.name == v_allele), None)
d_allele = next((allele for family in data_config_builtin.d_alleles.values() for allele in family if allele.name == d_allele), None)
j_allele = next((allele for family in data_config_builtin.j_alleles.values() for allele in family if allele.name == j_allele), None)
c_allele =  next((allele for family in data_config_builtin.c_alleles.values() for allele in family if allele.name == c_allele), None)

# Check if all alleles were found
if not v_allele or not d_allele or not j_allele:
    raise ValueError("One or more specified alleles could not be found in the data config.")


# Generate a sequence with the specified allele combination
specific_allele_sequence = HeavyChainSequence([v_allele, d_allele, j_allele,c_allele], data_config_builtin)
specific_allele_sequence.mutate(s5f_model)



print("Specific Allele Combination Sequence:", specific_allele_sequence.mutated_seq)


Specific Allele Combination Sequence: CAGGTGCAGGTGGTGCAGTATGGGGCTGAGGTGAAGAAGCCTGGGTCCGCGGTGAAGGTCTCCTGCAAGGCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGAGGGATCATCCCTATCTTTGGTACAGCCAACTACGCACAGAAGTTCCAGGACAGAGTCACGATTACCGCGGGTGAATCCACGAGCACAGCCTACATGGAGCTGAGCAGCCTGAGATCTGAGGACACGGCCGTGTATTACTGTGCGAGAGATCGGAAGCTACTACTACTACATGGACGTCTGGGGCAAAGGGACCACGGTCACCGTCTCCTCAGGCAGCCC


### Generating Naïve vs. Mutated Sequence Pairs

Comparing naïve and mutated versions of the same sequence can be useful for studying somatic hypermutation effects. Here's how to generate such pairs with GenAIRR.


In [15]:
# Generate a naive sequence
sequence_object = HeavyChainSequence.create_random(data_config_builtin)
sequence_object.mutate(s5f_model)

print("Naïve Sequence:", sequence_object.ungapped_seq)
print("Mutated Sequence:", sequence_object.mutated_seq)


Naïve Sequence: GAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCTGGGGGGTCCCTGAAACTCTCCTGTGCAGCCTCTGGGTTCACCTTCAGTGGCTCTGCTATGCACTGGGTCCGCCAGGCTTCCGGGAAAGGGCTGGAGTGGGTTGGCCGTATTAGAAGCAAAGCTAACAGTTACGCGACAGCATATGCTGCGTCGGTGAAAGGCAGGTTCACCATCTCCAGAGATGATTCAAAGAACACGGCGTATCTGCAAATGAACAGCCTGAAAACCGAGGACACGGCCGTGTATTACTGTACTAGCTTGGGGGTTATAGCAGCGGCCGGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGCTTCCACCAAGGGCC
Mutated Sequence: GAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCTGGGGGGTCCCTGAAACTCTCCTGTGCAGCCTCTGGGTTAACCTTCAGTGGCTCTGCTATGCACTGGGTCCGCCAGGCTTCCGAGAAAGGGCTGGAGTGGGTTGGCCGTATTAGAAGCAAAGCTAACAGTTACGCGACAGCATCTGCTGCGTCGGTGAAAGGCAGGTTCACCATCTCCAGAGATGATTCAAAGAACACGGCGTATCTGCAAATGAACAGCCTGAAAACCGAGGACACGGCCGTGTCTTATTGTACTAGGTCGGGGGCTATAGAAGCGGCCCGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGCTGCAACCAAGGGCC


## Conclusion

This section highlighted some common use cases for GenAIRR, demonstrating its flexibility in simulating AIRR sequences for various research purposes. Whether you need large datasets, specific allele combinations, custom mutation rates, or comparative analyses of naïve and mutated sequences, GenAIRR provides the necessary tools to achieve your objectives.
